In [ ]:
import pygame
import numpy as np
import sys
import math

#초깃값 설정
pygame.init()
height, width = 600, 800
screen = pygame.display.set_mode((width, height))
clock = pygame.time.Clock()

# 정육면체 꼭짓점 정의
cube_vertices = np.array([
    [-1, -1, -1],
    [-1, -1,  1],
    [-1,  1, -1],
    [-1,  1,  1],
    [ 1, -1, -1],
    [ 1, -1,  1],
    [ 1,  1, -1],
    [ 1,  1,  1]   
])

# 정육면체 모서리 정의
edges = [
    (0, 1), (1, 3), (3, 2), (2, 0),
    (4, 5), (5, 7), (7, 6), (6, 4),
    (0, 4), (1, 5), (2, 6), (3, 7) 
]

# 회전 행렬 함수 정의
def rotate_x(angle):
    cos_theta, sin_theta = math.cos(angle), math.sin(angle)
    return np.array([
        [1, 0, 0],
        [0, cos_theta, -sin_theta],
        [0, sin_theta, cos_theta]
    ])

def rotate_y(angle):
    cos_theta, sin_theta = math.cos(angle), math.sin(angle)
    return np.array([
        [cos_theta, 0, sin_theta],
        [0, 1, 0],
        [-sin_theta, 0, cos_theta]
    ])

def rotate_z(angle):
    cos_theta, sin_theta = math.cos(angle), math.sin(angle)
    return np.array([
        [cos_theta, -sin_theta, 0],
        [sin_theta, cos_theta, 0],
        [0, 0, 1]
    ])

camera_distance = 5.0
camera_pos = np.array([0.0, 0.0, -camera_distance])
camera_target = np.array([0.0, 0.0, 0.0])
camera_up = np.array([0.0, 1.0, 0.0])

# 뷰 행렬 계산
def look_at(camera_pos, camera_target, camera_up):
    forward = camera_target - camera_pos
    forward /= np.linalg.norm(forward)
    
    right = np.cross(camera_up, forward)
    right /= np.linalg.norm(right)
    
    up = np.cross(forward, right)
    mat = np.array([right, up, forward])
    return mat

# 3D -> 2D 투영
def project(point3d, distance):
    factor = 500 / (distance + point3d[2]) #거리(줌)
    x = point3d[0] * factor + width // 2
    y = -point3d[1] * factor + height // 2
    return (int(x), int(y))

angle_x = angle_y = angle_z = 0

while True:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()  
            
    if event.type == pygame.MOUSEWHEEL:
        camera_distance -= event.y * 0.5
        camera_distance = max(1, min(20, camera_distance)) #1~20
        camera_pos = np.array([0, 0, -camera_distance]) #z축 정보
        
    # 마우스 위치를 기반으로 회전(x, y 좌표)
    mx, my = pygame.mouse.get_pos()
    angle_x = (mx - height // 2) * 0.005
    angle_y = (mx - width // 2) * 0.005
    
    # 회전 및 카메라 뷰 값 구하기(변환)
    rotation = rotate_x(angle_x) @ rotate_y(angle_y) @ rotate_z(angle_z)
    view_matrix = look_at(camera_pos, camera_target, camera_up)
    
    transformed_vertices = []
    for v in cube_vertices: # 큐브의 각 꼭짓점을 돌면서
        rotated = rotation @ v # 각 꼭짓점이 얼마나 회전했는가를 구함
        camera_space = view_matrix @ (rotated - camera_pos) # 화면과 물체를 계산
        projeced = project(camera_space, camera_distance) # 화면에서 3D 정보를 2D로 보이게 투영
        
        transformed_vertices.append(projeced) # 투영된 좌표를 리스트에 담아서
    
    screen.fill((0, 0, 0))
    for edge in edges:
        pygame.draw.line(screen, (255, 255, 255), transformed_vertices[edge[0]], transformed_vertices[edge[1]], 2)
    
    pygame.display.flip()
    clock.tick(60)